In [ ]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'tax-index'
vectorstore = PineconeVectorStore.from_existing(
    index_name=index_name,
    embedding=embedding,)

llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}

    질문: {{question}}

""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain




In [ ]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인 소득세는 얼마인가?"

In [ ]:
retriever.invoke(query)

[Document(id='32bb69bc-b478-4a68-aa5c-d7182f449106', metadata={'source': './data/tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제

In [53]:
ai_message = qa_chain.invoke({"query": query})

In [54]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '연봉 5천만 원인 직장인의 소득세는 근로소득에 적용되는 기본세율에 따라 결정됩니다. 기본세율은 누진세율로, 세율과 소득구간에 따라 달라집니다. 연봉 및 공제 사항에 따라 달라질 수 있으니, 정확한 세액을 계산하기 위해서는 구체적인 세율표 및 공제 항목을 참고하는 것이 필요합니다.'}

In [ ]:
# new_question = dictionary_chain.invoke({"question": query})
# new_question

'질문: 연봉 5천만원 거주자 소득세는 얼마인가?'

In [ ]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가?"
query = "연봉 5천만원 직장인의 소득세는 얼마인가?"

In [66]:
ai_message = tax_chain.invoke({"question": query})

In [67]:
ai_message

{'query': '연봉 5천만원 거주자의 소득세는 얼마인가?',
 'result': '연봉 5천만원 거주자의 소득세는 624만원입니다. 이는 과세표준 5천만원을 초과하지 않으므로 해당 구간의 세율을 적용하여 계산한 금액입니다.'}